In [ ]:
import pandas as pd
import numpy as np
import keras
import math

In [86]:
def reformat_input(x):
    x = x.reshape((
        x.shape[0],
        math.isqrt(x.shape[1]),
        -1
    ))
    x = x.astype("float32") / 255
    x = np.expand_dims(x, -1)
    return x

In [89]:
train_data = pd.read_csv("train.csv")

x_train = train_data.drop('label', axis=1).to_numpy()
x_train = reformat_input(x_train)
y_train = train_data['label']

In [90]:
test_data = pd.read_csv("test.csv")

x_test = test_data.to_numpy()
x_test = reformat_input(x_test)

In [93]:
num_classes = 10
input_shape = x_train[0].shape

model = keras.Sequential(
    [
        keras.layers.Input(shape=input_shape),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(num_classes, activation="softmax"),
    ]
)

In [94]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 26, 26, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 24, 24, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 10, 10, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 8, 8, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 260,298 (1016.79 KB)

 Trainable params: 260,298 (1016.79 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=[
        keras.metrics.SparseCategoricalAccuracy(name="acc"),
    ],
)

In [ ]:
batch_size = 128
epochs = 20

callbacks = [
    keras.callbacks.ModelCheckpoint(filepath="model_at_epoch_{epoch}.keras"),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=2),
]

model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.15,
    callbacks=callbacks,
)

Epoch 1/20
278/279 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - acc: 0.4262 - loss: 1.5602

2024-11-28 12:43:01.592973: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{} for conv (f32[116,64,24,24]{3,2,1,0}, u8[0]{0}) custom-call(f32[116,64,26,26]{3,2,1,0}, f32[64,64,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}
2024-11-28 12:43:01.886192: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{} for conv (f32[116,128,10,10]{3,2,1,0}, u8[0]{0}) custom-call(f32[116,64,12,12]{3,2,1,0}, f32[128,64,3,3]{3,2,1,0}, f32[128]{0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={

279/279 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - acc: 0.4270 - loss: 1.5581

2024-11-28 12:43:05.948620: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{} for conv (f32[28,64,24,24]{3,2,1,0}, u8[0]{0}) custom-call(f32[28,64,26,26]{3,2,1,0}, f32[64,64,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kRelu","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}
2024-11-28 12:43:06.087206: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{} for conv (f32[28,128,10,10]{3,2,1,0}, u8[0]{0}) custom-call(f32[28,64,12,12]{3,2,1,0}, f32[128,64,3,3]{3,2,1,0}, f32[128]{0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cud

279/279 ━━━━━━━━━━━━━━━━━━━━ 16s 44ms/step - acc: 0.4278 - loss: 1.5560 - val_acc: 0.9370 - val_loss: 0.2274
Epoch 2/20
279/279 ━━━━━━━━━━━━━━━━━━━━ 7s 27ms/step - acc: 0.9046 - loss: 0.3143 - val_acc: 0.9668 - val_loss: 0.1213
Epoch 3/20
279/279 ━━━━━━━━━━━━━━━━━━━━ 8s 28ms/step - acc: 0.9396 - loss: 0.1991 - val_acc: 0.9725 - val_loss: 0.0963
Epoch 4/20
279/279 ━━━━━━━━━━━━━━━━━━━━ 8s 29ms/step - acc: 0.9529 - loss: 0.1600 - val_acc: 0.9706 - val_loss: 0.1028
Epoch 5/20
279/279 ━━━━━━━━━━━━━━━━━━━━ 8s 29ms/step - acc: 0.9617 - loss: 0.1288 - val_acc: 0.9786 - val_loss: 0.0727
Epoch 6/20
279/279 ━━━━━━━━━━━━━━━━━━━━ 8s 29ms/step - acc: 0.9659 - loss: 0.1149 - val_acc: 0.9846 - val_loss: 0.0568
Epoch 7/20
279/279 ━━━━━━━━━━━━━━━━━━━━ 8s 29ms/step - acc: 0.9704 - loss: 0.1002 - val_acc: 0.9840 - val_loss: 0.0544
Epoch 8/20
279/279 ━━━━━━━━━━━━━━━━━━━━ 8s 28ms/step - acc: 0.9728 - loss: 0.0895 - val_acc: 0.9870 - val_loss: 0.0464
Epoch 9/20
279/279 ━━━━━━━━━━━━━━━━━━━━ 8s 29ms/step - acc

ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 28000
'y' sizes: 4200


In [97]:
predictions = model.predict(x_test)

875/875 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step


In [98]:
output = np.array([
    np.array([i + 1, predictions[i].argmax()]) for i in range(len(predictions))
])

In [101]:
pd.DataFrame(data = output, columns=["ImageId", "Label"]).to_csv("output.csv", index=False)